### 1. 패키지 설치

In [ ]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-pinecone langchainhub

/Users/lhj/inflearn-langchain-rag/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


### 2. Knowledge Base 구성을 위한 데이터 생성
- split 된 데이터 chunk를 LLM에게 전달하면 토큰 절약 가능 -> 비용 감소와 답변 생성시간 감소의 효과
- LangChain에서 다양한 TextSplitter들을 제공
- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
    - chunk_size 는 split 된 chunk의 최대 크기
    - chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

#### * 이때, 임베딩이 잘 되기 위해서는 데이터 전처리가 필요하다.
- 데이터의 이미지를 텍스트로 변환해야하며
- 표 데이터인 경우 **마크다운**으로 변환해야한다. 

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('../tax_document/tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# Upstage 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [24]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

### 3. 관련 정보를 가져오는 Retrieval (검색)
- Chroma에 저장한 데이터에서 질문 쿼리와 관련있는 데이터를 가져오는 과정
- similarity_search(): 유사도 검색 제공 
- `k` 값을 조절해서 얼마나 많은 연관 데이터를 불러올지 결정

In [ ]:
query = '연봉 5천만원인 의 소득세는 얼마인가요?'

retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
retrieved_docs

[Document(id='b6cf13ea-d387-4463-aabf-2551c5d16ba7', metadata={'source': './tax_with_markdown.docx'}, page_content='⑧ 재해손실세액공제에 관하여 필요한 사항은 대통령령으로 정한다.\n\n[전문개정 2009. 12. 31.]\n\n[시행일: 2025. 1. 1.]  제58조제2항\n\n\n\n제59조(근로소득세액공제) ①근로소득이 있는 거주자에 대해서는 그 근로소득에 대한 종합소득산출세액에서 다음의 금액을 공제한다. <개정 2014. 1. 1., 2015. 5. 13.>\n\n\n\n② 제1항에도 불구하고 공제세액이 다음 각 호의 구분에 따른 금액을 초과하는 경우에 그 초과하는 금액은 없는 것으로 한다.<신설 2014. 1. 1., 2015. 5. 13., 2022. 12. 31.>\n\n1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n\n제59조의2(자녀세액공제) ①종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀(입양자 및 위탁아동을 포함하며, 이하 이 조에서 “

### 4. Augmentation(증강)을 위한 Prompt 활용
- Retrieval된 데이터를 프롬프트를 활용하여 LLM에 전달 
- LangChain에서 제공하는 프롬포트("rlm/rag-prompt")를 사용해도 되고, 본인이 직접 작성해도 된다. 

In [6]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

context = "\n\n---".join([doc.page_content for doc in retrieved_docs])

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 최고의 한국 소득세 전문가입니다. 
                주어진 context를 기반으로 질문에 답변하세요."""
    ),
    ("human", """Context: {context}
                Question: {question}"""
    )
])

chat_value = rag_prompt.invoke({"context": context, "question": query})

chat_value

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 최고의 한국 소득세 전문가입니다. \n                주어진 context를 기반으로 질문에 답변하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Context: {context}\n                Question: {question}'), additional_kwargs={})])

### 5. 답변 생성 (Generation)
- 프롬포트를 LLM에게 전달하여 답변 생성 

In [20]:
# 7. LLM 호출
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [21]:
ai_response = llm.invoke(chat_value)

print(ai_response.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해 다음 단계를 따라야 합니다.

1. **총급여액 확인**: 연봉이 5천만원입니다.

2. **근로소득세액공제 확인**: 
   - 총급여액이 4천 800만원(3천 300만원 초과 7천만원 이하)에 해당하므로, 공제금액을 다음과 같이 계산합니다.
   - 공제액 = 74만원 - [(5천만원 - 3천 300만원) × 8/1000]
   - = 74만원 - [(1천 700만원) × 0.008]
   - = 74만원 - 13.6만원
   - = 60.4만원이지만, 66만원보다 적은 경우에는 66만원이 최종 공제금액입니다. 따라서, **66만원**을 공제받습니다.

3. **소득세 계산**:
   - 총급여액 5천만원에 대한 종합소득 과세표준은 5천만원입니다.
   - 세율은 다음과 같이 적용됩니다.
     - 1,400만원 이하: 1,400만원 × 6% = 84만원
     - 1,400만원 초과 5,000만원 이하: 84만원 + [(5,000만원 - 1,400만원) × 15%]
     - = 84만원 + [(3,600만원) × 0.15]
     - = 84만원 + 540만원
     - = 624만원

4. **최종 세액에서 공제 적용**:
   - 최종적으로 계산된 세액은 624만원에서 공제금액 66만원을 빼면 됩니다.
   - 최종 세액 = 624만원 - 66만원 = 558만원

따라서 연봉 5천만원인 직장인의 소득세는 **558만원**입니다.
